In [2]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from pyvi.ViTokenizer import tokenize

RANDOM_STATE = 42

In [3]:
model = SentenceTransformer("all-mpnet-base-v2")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [4]:
game_df = pd.read_csv("video_games.csv")
game_df.head(5)

,name,platform,release_date,summary,user_review
0,The Legend of Zelda: Ocarina of Time,Nintendo 64,"November 23, 1998","As a young boy, Link is tricked by Ganondorf, ...",9.1
1,Tony Hawk's Pro Skater 2,PlayStation,"September 20, 2000",As most major publishers' development efforts ...,7.4
2,Grand Theft Auto IV,PlayStation 3,"April 29, 2008",[Metacritic's 2008 PS3 Game of the Year; Also ...,7.7
3,SoulCalibur,Dreamcast,"September 8, 1999","This is a tale of souls and swords, transcendi...",8.4
4,Grand Theft Auto IV,Xbox 360,"April 29, 2008",[Metacritic's 2008 Xbox 360 Game of the Year; ...,7.9


In [5]:
game_df.drop_duplicates("name", inplace=True)
game_description = game_df[["name", "summary"]].head(50)
pd.set_option('display.max_colwidth', None)
game_description

,name,summary
0,The Legend of Zelda: Ocarina of Time,"As a young boy, Link is tricked by Ganondorf, the King of the Gerudo Thieves. The evil human uses Link to gain access to the Sacred Realm, where he places his tainted hands on Triforce and transforms the beautiful Hyrulean landscape into a barren wasteland. Link is determined to fix the problems he helped to create, so with the help of Rauru he travels through time gathering the powers of the Seven Sages."
1,Tony Hawk's Pro Skater 2,"As most major publishers' development efforts shift to any number of next-generation platforms, Tony Hawk 2 will likely stand as one of the last truly fantastic games to be released on the PlayStation."
2,Grand Theft Auto IV,"[Metacritic's 2008 PS3 Game of the Year; Also known as ""GTA IV""] What does the American Dream mean today? For Niko Belic, fresh off the boat from Europe. It's the hope he can escape his past. For his cousin, Roman, it is the vision that together they can find fortune in Liberty City, gateway to the land of opportunity. As they slip into debt and are dragged into a criminal underworld by a series of shysters, thieves and sociopaths, they discover that the reality is very different from the dream in a city that worships money and status, and is heaven for those who have them an a living nightmare for those who don't. [Rockstar Games]"
3,SoulCalibur,"This is a tale of souls and swords, transcending the world and all its history, told for all eternity... The greatest weapons-based fighter returns, this time on Sega Dreamcast. Soul Calibur unleashes incredible graphics, fantastic fighters, and combos so amazing they'll make your head spin!"
5,Super Mario Galaxy,"[Metacritic's 2007 Wii Game of the Year] The ultimate Nintendo hero is taking the ultimate step ... out into space. Join Mario as he ushers in a new era of video games, defying gravity across all the planets in the galaxy. When some creature escapes into space with Princess Peach, Mario gives chase, exploring bizarre planets all across the galaxy. Mario, Peach and enemies new and old are here. Players run, jump and battle enemies as they explore all the planets in the galaxy. Since this game makes full use of all the features of the Wii Remote, players have to do all kinds of things to succeed: pressing buttons, swinging the Wii Remote and the Nunchuk, and even pointing at and dragging things with the pointer. Since he's in space, Mario can perform mind-bending jumps unlike anything he's done before. He'll also have a wealth of new moves that are all based around tilting, pointing and shaking the Wii Remote. Shake, tilt and point! Mario takes advantage of all the unique aspects of the Wii Remote and Nunchuk controller, unleashing new moves as players shake the controller and even point at and drag items with the pointer. [Nintendo]"
6,Super Mario Galaxy 2,"Super Mario Galaxy 2, the sequel to the galaxy-hopping original game, includes the gravity-defying, physics-based exploration from the first game, but is loaded with entirely new galaxies and features to challenge players. On some stages, Mario can pair up with his dinosaur buddy Yoshi and use his tongue to grab items and spit them back at enemies. Players can also have fun with new items such as a drill that lets our hero tunnel through solid rock. [Nintendo]"
7,Red Dead Redemption 2,"Developed by the creators of Grand Theft Auto V and Red Dead Redemption, Red Dead Redemption 2 is an epic tale of life in America’s unforgiving heartland. The game’s vast and atmospheric world also provides the foundation for a brand new online multiplayer experience. America, 1899. The end of the Wild West era has begun. After a robbery goes badly wrong in the western town of Blackwater, Arthur Morgan and the Van der Linde gang are forced to flee. With federal agents and the best bounty hunters in the nation massing on their heels, the gang has to rob, steal and fight their way across the rugged heartland of America in order to survi

In [ ]:
game_description["tokenize_sum"] = game_description["summary"].apply(tokenize)

In [ ]:
embeddings = model.encode(game_description["tokenize_sum"].to_numpy())

In [ ]:
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
import numpy as np
import pandas as pd
import plotly.express as px

def clustering(embeddings, num_clusters):
    model = KMeans(n_clusters=num_clusters)
    model.fit(embeddings)
    return model.labels_

In [ ]:
def visualization(descriptions, embeddings, n_cluster=6):
    X = np.array(embeddings)
    pca = PCA(n_components=3)
    result = pca.fit_transform(X)
    df = pd.DataFrame({
    'sent': descriptions["name"],
    'cluster': clustering(embeddings, n_cluster).astype(str),
    'x': result[:, 0],
    'y': result[:, 1],
    'z': result[:, 2]
    })
    fig = px.scatter_3d(df, x='x', y='y', z='z',
              color='cluster', hover_name='sent',
              range_x = [df.x.min()-0.5, df.x.max()+0.5],
              range_y = [df.y.min()-0.5, df.y.max()+0.5],
              range_z = [df.z.min()-0.5, df.z.max()+0.5])
    fig.update_layout(
    width=1280,   # Set the desired width
    height=720   # Set the desired height
    )
    fig.update_traces(hovertemplate= '<b>%{hovertext}</b>')
    fig.show()

In [ ]:
visualization(game_description, embeddings, n_cluster=4)

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
for i in game_description[(game_description["name"] == "Tony Hawk's Pro Skater 3") | (game_description["name"] == "Tony Hawk's Pro Skater 2")]["summary"]:
    print(i)

In [ ]:
test_df = game_description[(game_description["name"] == "Tony Hawk's Pro Skater 3") | (game_description["name"] == "Tony Hawk's Pro Skater 2")]

enc_L = model.encode(test_df["tokenize_sum"].to_numpy())
cosine_similarity([enc_L[0]], [enc_L[1]])